In [1]:
!pip3 install transformers
from transformers import AutoTokenizer
tokenizer_path = 'facebook/opt-350m'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
!huggingface-cli login --token REPLACE WITH TOKEN
!pip3 install dotenv
import os 
from dotenv import load_dotenv
import huggingface_hub

# Load environment variables (make sure .env contains HF_TOKEN)
load_dotenv()
hf_token = os.getenv("HF_TOKEN")
# Set the repo URL or name (must be under your HF account or organization)
repo_name = "Talking-Babies"


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `climb` has been saved to /Users/suchirsalhan/.cache/huggingface/stored_tokens
Your token has been saved to /Users/suchirsalhan/.cache/huggingface/token
Login successful.
The current active token is: `climb`

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip


In [4]:
!pip3 install datasets
!pip3 install Jinja2
from transformers import AutoTokenizer
from datasets import load_dataset
from dotenv import load_dotenv
import os

# Load Hugging Face token from .env
load_dotenv()
hf_token = os.getenv("HF_TOKEN")

# Load the base tokenizer (same as before)
tokenizer_path = 'facebook/opt-350m'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

# Load HF dataset
print("Loading dataset...")
dataset = load_dataset("tafseer-nayeem/KidLM-corpus", split="train")

# Define data iterator over text field
def data_iterator():
    print("Creating a generator over the KidLM corpus...")
    for example in dataset:
        yield example["text"]

# Train a new tokenizer with vocab size 16,000
print("Training new tokenizer with 16k vocabulary size -- keeping all other settings the same")
new_tokenizer = tokenizer.train_new_from_iterator(data_iterator(), vocab_size=16000)

# Push to hub
print("Pushing tokenizer to Hugging Face Hub...")
new_tokenizer.push_to_hub("Talking-Babies", use_auth_token=hf_token)


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Loading dataset...
Training new tokenizer with 16k vocabulary size -- keeping all other settings the same
Creating a generator over the KidLM corpus...



Pushing tokenizer to Hugging Face Hub...


CommitInfo(commit_url='https://huggingface.co/suchirsalhan/Talking-Babies/commit/85bc004c6ceb4ea1f83d9bda2b3e7e20e44e3a6b', commit_message='Upload tokenizer', commit_description='', oid='85bc004c6ceb4ea1f83d9bda2b3e7e20e44e3a6b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/suchirsalhan/Talking-Babies', endpoint='https://huggingface.co', repo_type='model', repo_id='suchirsalhan/Talking-Babies'), pr_revision=None, pr_num=None)

OPT Tokenizer for KidLM  – if we want to do anything for fixed seq lengths

In [ ]:
import os
from dotenv import load_dotenv
from transformers import AutoTokenizer

load_dotenv()

HF_TOKEN = os.getenv("HF_TOKEN")
tokenizer = AutoTokenizer.from_pretrained("Talking-Babies/opt-tokenizer")

In [ ]:
def tokenize_and_chunk(examples, seq_len):
    """
    Tokenizes and chunks text data to fixed-length sequences.
    
    Args:
        examples: A batch of text examples from the dataset
        seq_len: The length of the sequences to chunk the text into
        
    Returns:
        Dictionary containing chunked token sequences of length SEQ_LEN
    """
    tokens = []
    # Process each text example in the batch
    for text in examples['text']:
        # Convert text to token IDs
        _tokens = tokenizer.encode(text)
        # Add EOS token to mark the end of each text example
        _tokens.append(tokenizer.eos_token_id)
        # Accumulate all tokens in a flat list
        tokens.extend(_tokens)

    # Split the accumulated tokens into chunks of SEQ_LEN
    chunks = [tokens[i:i + seq_len] for i in range(0, len(tokens), seq_len)]
    
    # Discard the last chunk if it's shorter than SEQ_LEN to ensure uniform sequence length
    if len(chunks[-1]) < seq_len:
        chunks = chunks[:-1]
        
    return {'input_ids': chunks}

In [ ]:
from datasets import Dataset
import os
data_root_path = 'data/raw/train_100M'

raw_data_list= []

def raw_data_iterator(data_root_path):
    for file in os.listdir(data_root_path):
        print("Processing file: ", file)
        with open(os.path.join(data_root_path, file), 'r') as f:
            document = []
            for line in f:
                if line == '\n':
                    raw_data_list.append({'text': ' '.join(document)})
                    document = []
                else:
                    document.append(line)

data_iterator = raw_data_iterator(data_root_path)
raw_dataset = Dataset.from_list(raw_data_list)
raw_dataset = raw_dataset.shuffle(seed=420)
from huggingface_hub import HfApi
api = HfApi()
from functools import partial
SINGLE_SHUFFLE=False

In [ ]:
tokenize_and_chunk_2048 = partial(tokenize_and_chunk, seq_len=2048)

tokenized_dataset_2048 = raw_dataset.map(
    tokenize_and_chunk_2048,
    batched=True,
    batch_size=500,
    num_proc=8,
    remove_columns=raw_dataset.column_names
)
if not SINGLE_SHUFFLE:
    tokenized_dataset_2048 = tokenized_dataset_2048.shuffle(seed=42)

In [ ]:
parquet_path = 'data/processed/train_100M_2048'
if SINGLE_SHUFFLE:
    parquet_path += '_single_shuffle'
parquet_path += '.parquet'
tokenized_dataset_2048.to_parquet(parquet_path)

In [ ]:
repo_id = "babylm-seqlen/train_100M_2048"   
if SINGLE_SHUFFLE:
    repo_id += "_single_shuffle"
api.create_repo(repo_id, private=False, exist_ok=True, token=HF_TOKEN, repo_type="dataset")

api.upload_file(
    path_or_fileobj=parquet_path,
    repo_id=repo_id,
    repo_type='dataset',
    path_in_repo='train_100M_2048_single_shuffle.parquet' if SINGLE_SHUFFLE else 'train_100M_2048.parquet',
    token=HF_TOKEN
)